<a href="https://colab.research.google.com/github/Artur471/missp/blob/master/pulp/whiskas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:


In [2]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

['jabłko:5', 'banan:10']
jabłko:5, banan:10


In [5]:
from pulp import *

gatunki_mies = ["kurczak", "wolowina", "baranina"]

prob = LpProblem("The Whiskas Problem",LpMinimize)

x1 = LpVariable("ChickenPercent", 0)
x2 = LpVariable("BeefPercent", 0)
x3 = LpVariable("MuttonPercent",0)

prob += 0.013*x1 + 0.008*x2 + 0.009*x3, "Total Cost of Ingreduents per can"

prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.250*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.050*x3 >= 6.0, "FatRequirement"
prob += 0.001*x2 + 0.005*x2 + 0.002*x3 <= 2.0, "FiberRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.004*x3 <= 0.4, "SaltRequirement"
prob += 0.0003*x1 + 0.0026*x2 + 0.0026*x2 >= 0.0008, "VitaminB_Requriment"

prob.writeLP("WhiskasModel.lp")
prob.solve()

print ("Status:", LpStatus[prob.status])

for v in prob.variables():
  print (v.name, "=", v.varValue)

podsumowanie_zawartosc = [str(a.varValue) for a in prob.objective]

podsumowanie_wyniku = ["% ".join((a, b)) for a, b in zip(podsumowanie_zawartosc, gatunki_mies)]
podsumowanie_wyniku = ", ".join(podsumowanie_wyniku)

print("Total Cost of Ingredients per can = ", value(prob.objective))
print("Końcowy produkt będzie kosztować {}, będzie miał w składzie: {}".format(value(prob.objective), podsumowanie_wyniku))

Status: Optimal
BeefPercent = 15.384615
ChickenPercent = 7.6923077
MuttonPercent = 76.923077
Total Cost of Ingredients per can =  0.9153846131000001
Końcowy produkt będzie kosztować 0.9153846131000001, będzie miał w składzie: 7.6923077% kurczak, 15.384615% wolowina, 76.923077% baranina


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [6]:
from pulp import *
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'PEAR']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'PEAR': 0.0005}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'PEAR': 0.005}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'PEAR': 0.001}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'PEAR': 0.200}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'PEAR': 0.0001}

vitaminB_Percent = {'CHICKEN': 0.001, 
               'BEEF': 0.003, 
               'MUTTON': 0.002, 
               'RICE': 0.004, 
               'WHEAT': 0.003, 
               'GEL': 0.000,
               'PEAR': 0.25}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)

# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)

# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminB_Percent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.0008, "VitaminB_Requriment"

prob.writeLP("WhiskasModel.lp")
prob.solve()

print ("Status:", LpStatus[prob.status])

for v in prob.variables():
  print (v.name, "=", v.varValue)

podsumowanie_zawartosc = [str(a.varValue) for a in prob.objective]
print(podsumowanie_zawartosc)
podsumowanie_wyniku = ["% ".join((a, b)) for a, b in zip(podsumowanie_zawartosc, Ingredients)]
podsumowanie_wyniku = ", ".join(podsumowanie_wyniku)
print("Total Cost of Ingredients per can = ", value(prob.objective))
print("Końcowy produkt będzie kosztować {}, będzie miał w składzie: {}".format(value(prob.objective), podsumowanie_wyniku))


Status: Optimal
Ingr_BEEF = 59.914979
Ingr_CHICKEN = 0.0
Ingr_GEL = 31.582896
Ingr_MUTTON = 0.0
Ingr_PEAR = 8.5021255
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0
['0.0', '59.914979', '0.0', '0.0', '0.0', '31.582896', '8.5021255']
Total Cost of Ingredients per can =  0.51515379075
Końcowy produkt będzie kosztować 0.51515379075, będzie miał w składzie: 0.0% CHICKEN, 59.914979% BEEF, 0.0% MUTTON, 0.0% RICE, 0.0% WHEAT, 31.582896% GEL, 8.5021255% PEAR


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
